In [198]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from prince import MCA  # Multiple Correspondence Analysis for categorical data
from kmodes.kprototypes import KPrototypes  # Best for mixed data clustering
from sklearn.metrics import silhouette_score
from tqdm import tqdm

In [199]:
# Load data
wave7_csv = "data/preprocessed/cleaned_wave_7_rq2.csv"
df = pd.read_csv(wave7_csv, low_memory=False)
df.head()

,Country,C Television,C Police,C Courts,C Government,C Political parties,C Civil services,C Elections,Importance of democracy,Highest educational level,Age,Scale of incomes,Sex,Strong Leader,Expert Non Govt Person,Signing a petition,Joining unofficial strikes
0,20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,8.0,3.0,60.0,5.0,2.0,4.0,4.0,2.0,3.0
1,20,3.0,3.0,3.0,4.0,4.0,3.0,3.0,10.0,7.0,47.0,9.0,1.0,4.0,4.0,1.0,2.0
2,20,4.0,2.0,2.0,3.0,3.0,3.0,3.0,10.0,7.0,48.0,5.0,1.0,4.0,2.0,1.0,2.0
3,20,3.0,3.0,3.0,3.0,3.0,3.0,3.0,7.0,2.0,62.0,4.0,2.0,3.0,2.0,2.0,2.0
4,20,3.0,2.0,2.0,2.0,3.0,3.0,3.0,8.0,2.0,49.0,4.0,1.0,3.0,3.0,1.0,1.0


In [200]:
for col in df.columns:
    print(col)

Country
C Television
C Police
C Courts
C Government
C Political parties
C Civil services
C Elections
Importance of democracy
Highest educational level
Age
Scale of incomes
Sex
Strong Leader
Expert Non Govt Person
Signing a petition
Joining unofficial strikes


In [201]:
print(df["Country"].value_counts())

Country
124    4018
360    3200
156    3036
826    2609
840    2596
       ... 
32     1003
858    1000
152    1000
196    1000
909     447
Name: count, Length: 66, dtype: int64


In [202]:
# Define G8 country mapping
country_map = {
    840: "USA", 124: "Canada", 250: "France", 276: "Germany",
    380: "Italy", 392: "Japan", 643: "Russia", 826: "UK"
}

# Filter dataset for G8 countries
df_g8 = df[df["Country"].isin(country_map.keys())].copy()

# Map country codes to country names
df_g8["Country"] = df_g8["Country"].map(country_map)

# Verify data after filtering
print(df_g8["Country"].unique())

['Canada' 'Germany' 'UK' 'Japan' 'Russia' 'USA']


In [203]:
print(df_g8["Country"].value_counts())

Country
Canada     4018
UK         2609
USA        2596
Russia     1810
Germany    1528
Japan      1353
Name: count, dtype: int64


In [204]:
df_g8.head()

,Country,C Television,C Police,C Courts,C Government,C Political parties,C Civil services,C Elections,Importance of democracy,Highest educational level,Age,Scale of incomes,Sex,Strong Leader,Expert Non Govt Person,Signing a petition,Joining unofficial strikes
10072,Canada,3.0,2.0,3.0,2.0,3.0,2.0,2.0,10.0,7.0,76.0,9.0,2.0,4.0,3.0,1.0,2.0
10073,Canada,2.0,2.0,2.0,1.0,3.0,2.0,1.0,10.0,6.0,69.0,3.0,1.0,4.0,2.0,1.0,1.0
10074,Canada,3.0,3.0,3.0,3.0,3.0,3.0,3.0,5.0,3.0,35.0,3.0,2.0,2.0,2.0,3.0,3.0
10075,Canada,3.0,2.0,3.0,3.0,3.0,3.0,2.0,8.0,7.0,45.0,7.0,1.0,3.0,2.0,2.0,3.0
10076,Canada,3.0,2.0,4.0,4.0,4.0,3.0,3.0,10.0,6.0,32.0,5.0,1.0,4.0,3.0,1.0,1.0


In [205]:
# Ensure numeric columns only (exclude 'Country')
numeric_columns = df_g8.select_dtypes(include=[np.number]).columns

# Find negative values in numeric columns
negative_values = (df_g8[numeric_columns] < 0).sum()

# Display columns with negative values
print("Columns with Negative Values and Their Counts:\n", negative_values[negative_values > 0])


Columns with Negative Values and Their Counts:
 Series([], dtype: int64)


In [206]:
missing_values = df_g8.isnull().sum()
print("Missing Values:\n", missing_values[missing_values > 0])

Missing Values:
 C Television                  181
C Police                      154
C Courts                      418
C Government                  271
C Political parties           445
C Civil services              510
C Elections                   367
Importance of democracy       350
Highest educational level     254
Age                           140
Scale of incomes              771
Sex                            33
Strong Leader                 713
Expert Non Govt Person        893
Signing a petition            297
Joining unofficial strikes    561
dtype: int64


In [207]:
# Impute ordinal variables with mode (most frequent value)
ordinal_cols = ["C Television", "C Police", "C Courts", "C Government", 
                "C Political parties", "C Civil services", "C Elections", 
                "Importance of democracy", "Scale of incomes", 
                "Strong Leader", "Expert Non Govt Person"]

for col in ordinal_cols:
    df_g8[col] = df_g8[col].fillna(df_g8[col].mode()[0])

In [208]:
# Impute Age with median
df_g8["Age"] = df_g8["Age"].fillna(df_g8["Age"].median())

In [209]:
# Impute categorical variables with mode
categorical_cols = ["Highest educational level", "Sex", "Signing a petition", "Joining unofficial strikes"]

for col in categorical_cols:
    df_g8[col] = df_g8[col].fillna(df_g8[col].mode()[0])

In [210]:
# Drop variables with >50% missing values
df_g8.drop(columns=["Strong Leader", "Expert Non Govt Person", "Country"], inplace=True)

In [211]:
print("Remaining Missing Values:\n", df_g8.isnull().sum())

Remaining Missing Values:
 C Television                  0
C Police                      0
C Courts                      0
C Government                  0
C Political parties           0
C Civil services              0
C Elections                   0
Importance of democracy       0
Highest educational level     0
Age                           0
Scale of incomes              0
Sex                           0
Signing a petition            0
Joining unofficial strikes    0
dtype: int64


In [212]:
for col in df_g8.columns:
    print(col)

C Television
C Police
C Courts
C Government
C Political parties
C Civil services
C Elections
Importance of democracy
Highest educational level
Age
Scale of incomes
Sex
Signing a petition
Joining unofficial strikes


In [213]:
# ✅ Step 1: Define Variable Types
ordinal_columns = [
    "C Television", "C Police", "C Courts", "C Government",
    "C Political parties", "C Civil services", "C Elections",
    "Importance of democracy", "Scale of incomes"
]

categorical_columns = [
    "Highest educational level", "Sex", "Signing a petition", "Joining unofficial strikes"
]

continuous_columns = ["Age"]  # Only Age is continuous

# ✅ Step 2: Convert Ordinal Variables Properly
for col in ordinal_columns:
    df_g8[col] = pd.Categorical(df_g8[col], ordered=True).codes  # Convert to integer levels

# ✅ Step 3: Convert Categorical Variables using One-Hot Encoding
df_g8 = pd.get_dummies(df_g8, columns=categorical_columns, drop_first=True)

# ✅ Step 4: Normalize Continuous Variables
scaler = StandardScaler()
df_g8[continuous_columns] = scaler.fit_transform(df_g8[continuous_columns])

print("✅ Data Preprocessing Completed!")

✅ Data Preprocessing Completed!


In [ ]:
# import matplotlib.pyplot as plt

# # 🚀 Elbow Method for K-Prototypes
# cost = []
# K = range(2, 6)  # Testing cluster numbers from 2 to 5

# for num_clusters in tqdm(K, desc="Finding Optimal Clusters"):
#     kproto = KPrototypes(n_clusters=num_clusters, random_state=42)
#     clusters = kproto.fit_predict(df_g8, categorical=list(range(len(ordinal_columns), df_g8.shape[1])))
#     cost.append(kproto.cost_)  # Store cost

# # 🚀 Plot Elbow Graph
# plt.figure(figsize=(8,5))
# plt.plot(K, cost, marker="o", linestyle="-")
# plt.xlabel("Number of Clusters")
# plt.ylabel("Cost (Lower is Better)")
# plt.title("Elbow Method for Optimal K in K-Prototypes")
# plt.show()


In [ ]:
# 🚀 Define Number of Clusters (Based on Elbow Method)
num_clusters = 3  

# 🚀 Fit K-Prototypes Model
kproto = KPrototypes(n_clusters=num_clusters, random_state=42)
clusters = kproto.fit_predict(df_g8, categorical=list(range(len(ordinal_columns), df_g8.shape[1])))

# 🚀 Add Cluster Labels
df_g8["Cluster"] = clusters

print("✅ Clustering Completed. Cluster Counts:")
print(pd.Series(clusters).value_counts())

# 🚀 Save Clustered Data
df_g8.to_csv("data/preprocessed/clustered_rq4.csv", index=False)


In [ ]:
# 🚀 Cluster Profiling
cluster_summary = df_g8.groupby("Cluster").mean()
print("✅ Cluster Profiling Completed:")
print(cluster_summary)


In [ ]:
from prince import MCA

# 🚀 Fit MCA
mca = MCA(n_components=2, random_state=42)
mca_result = mca.fit_transform(df_g8.drop("Cluster", axis=1))

# 🚀 Convert to DataFrame
df_mca = pd.DataFrame(mca_result, columns=["MCA1", "MCA2"])
df_mca["Cluster"] = clusters

# 🚀 Plot MCA Clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x="MCA1", y="MCA2", hue=df_mca["Cluster"], palette="Set1", data=df_mca)
plt.title("MCA Representation of Clustering")
plt.xlabel("MCA Component 1")
plt.ylabel("MCA Component 2")
plt.legend(title="Cluster")
plt.show()
